In [1]:
import pandas as pd
import numpy as np
import netCDF4

In [51]:
# Load the csv file and correct the Release Time
fukurls = pd.read_csv('fukurls.csv')
fukurls.rename({'RTIME': 'ReleaseTime'}, axis=1, inplace=True)

def fix_rtime(t):
    s = str(t)
    return s[:3] + '0' + s[3:]

fukurls['ReleaseTime'] = fukurls['ReleaseTime'].apply(fix_rtime).astype('int64')
fukurls.set_index('ReleaseTime', inplace=True)

In [46]:
# Load the netcdf file into a DataFrame
ds = netCDF4.Dataset('puftrac_2018103100ChinShan_Cs.nc')
puftrac = pd.DataFrame(ds['trajec'][:])

In [52]:
fukurls.head()

,Cs(/h)
ReleaseTime,
1540944000,3.700080e+12
1540944300,3.700080e+12
1540944600,3.700080e+12
1540944900,3.700080e+12
1540945200,3.700080e+12


In [48]:
puftrac.head()

,ReleaseTime,CurrentTime,Longitude,Latitude,RelativeHeight,SigX,SigY,SigZ,AirConcentrationR,GroundConcentrationR
0,1540944000,1540944000,121.587265,25.286192,18.358713,20.881584,19.706356,18.560350,1.000000e+10,0.0
1,1540944000,1540944300,121.586586,25.285027,25.641741,235.425659,205.557571,97.498146,1.000000e+10,0.0
2,1540944300,1540944300,121.587303,25.286184,14.849241,26.482800,25.320646,24.167999,1.000000e+10,0.0
3,1540944000,1540944600,121.582283,25.280849,89.982407,439.083801,376.263611,130.351196,1.000000e+10,0.0
4,1540944300,1540944600,121.587730,25.284620,14.365055,240.414215,210.729919,100.204796,1.000000e+10,0.0


In [53]:
# Wherever ReleaseTime matches, scale AirConcentrationR and GroundConcentrationR with Cs(/h).

%%time

chvars = ['AirConcentrationR', 'GroundConcentrationR']

for rtime in fukurls.index:
    cond = puftrac['ReleaseTime'] == rtime
    puftrac.loc[cond, chvars] *= fukurls.loc[rtime, 'Cs(/h)']     

CPU times: user 48.2 s, sys: 0 ns, total: 48.2 s
Wall time: 48.1 s


In [68]:
# Convert DataFrame to structured numpy array
data_trajecout = np.array([tuple(r) for r in puftrac.values], dtype=ds['trajec'].dtype)

In [70]:
# Begin writing out changed `trajec` to disk
dsout = netCDF4.Dataset('puftrac_netcdf.nc', 'w')

In [71]:
stype = dsout.createCompoundType(ds['trajec'].dtype, 'stype')
dsout.createDimension('serial', len(ds['trajec']))
trajecout = dsout.createVariable('trajec', stype, ('serial',))
trajecout[:] = data_trajecout
for name in ds.ncattrs():
    setattr(dsout, name, getattr(ds, name, None))

In [72]:
dsout['trajec'][:]

array([ (1540944000, 1540944000,  121.58726501,  25.28619194,  18.35871315,   20.88158417,   19.70635605,  18.56035042,   3.70007999e+22,  0.),
       (1540944000, 1540944300,  121.586586  ,  25.28502655,  25.6417408 ,  235.42565918,  205.55757141,  97.49814606,   3.70007999e+22,  0.),
       (1540944300, 1540944300,  121.58730316,  25.28618431,  14.84924126,   26.48279953,   25.32064629,  24.16799927,   3.70007999e+22,  0.),
       ...,
       (1541634300, 1541635200,  121.58605957,  25.28359222,   0.8872261 ,  298.42910767,  311.3326416 ,  80.88805389,   1.00000000e+10,  0.),
       (1541634600, 1541635200,  121.58480835,  25.28500366,  56.78849411,  205.64807129,  217.37597656,  68.38516998,   1.00000000e+10,  0.),
       (1541634900, 1541635200,  121.58705902,  25.28547096,   1.60839462,  112.5715332 ,  117.04603577,  51.2441864 ,   1.00000000e+10,  0.)],
      dtype={'names':['ReleaseTime','CurrentTime','Longitude','Latitude','RelativeHeight','SigX','SigY','SigZ','AirConcentration

In [62]:
ds['trajec'][:]

array([ (1540944000, 1540944000,  121.58726501,  25.28619194,  18.35871315,   20.88158417,   19.70635605,  18.56035042,   1.00000000e+10,  0.),
       (1540944000, 1540944300,  121.586586  ,  25.28502655,  25.6417408 ,  235.42565918,  205.55757141,  97.49814606,   1.00000000e+10,  0.),
       (1540944300, 1540944300,  121.58730316,  25.28618431,  14.84924126,   26.48279953,   25.32064629,  24.16799927,   1.00000000e+10,  0.),
       ...,
       (1541634300, 1541635200,  121.58605957,  25.28359222,   0.8872261 ,  298.42910767,  311.3326416 ,  80.88805389,   1.00000000e+10,  0.),
       (1541634600, 1541635200,  121.58480835,  25.28500366,  56.78849411,  205.64807129,  217.37597656,  68.38516998,   1.00000000e+10,  0.),
       (1541634900, 1541635200,  121.58705902,  25.28547096,   1.60839462,  112.5715332 ,  117.04603577,  51.2441864 ,   1.00000000e+10,  0.)],
      dtype={'names':['ReleaseTime','CurrentTime','Longitude','Latitude','RelativeHeight','SigX','SigY','SigZ','AirConcentration

In [74]:
dsout.close()

In [75]:
ds.close()